In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense , Dropout, Flatten, MaxPooling2D , BatchNormalization , Conv2D 
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from keras.models import Sequential
from keras.applications import ResNet50

In [2]:
folder = r"F:\ML Python VsCode\Age_Gender_Emotion_Prediction_using_CNN\UTKFace"

In [3]:
age = []
gender = []
img = []
for filename in os.listdir(folder):
    age.append(int(filename.split('_')[0]))
    gender.append(int(filename.split('_')[1]))
    img.append(filename)

In [4]:
data = pd.DataFrame({'age':age,'gender':gender,'img':img})
data.head()

,age,gender,img
0,100,0,100_0_0_20170112213500903.jpg.chip.jpg
1,100,0,100_0_0_20170112215240346.jpg.chip.jpg
2,100,1,100_1_0_20170110183726390.jpg.chip.jpg
3,100,1,100_1_0_20170112213001988.jpg.chip.jpg
4,100,1,100_1_0_20170112213303693.jpg.chip.jpg


In [5]:
train_df = data.sample(frac=1,random_state=20).iloc[:20000]
test_df = data.sample(frac=1,random_state=20).iloc[20000:]

In [6]:
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
)
test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [7]:
train_gen = train_datagen.flow_from_dataframe(train_df,directory=folder,x_col='img', y_col='age',target_size=(200,200),batch_size=32,class_mode='raw')
test_gen = test_datagen.flow_from_dataframe(test_df,directory=folder,x_col='img',y_col='age',target_size=(200,200),batch_size=8,class_mode='raw')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [17]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    # Set TensorFlow to use only the GPU devices
    tf.config.experimental.set_visible_devices(gpus, 'GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)  #
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Age Model

In [10]:
model = Sequential()
Res = ResNet50()
model.add(Flatten())
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='relu'))

In [14]:
# from keras.utils import plot_model
# plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


  Using cached pydot-2.0.0-py3-none-any.whl.metadata (9.6 kB)
Using cached pydot-2.0.0-py3-none-any.whl (22 kB)


In [15]:
model.compile(loss='mae',optimizer='adam',metrics=['mae'])

In [16]:
model.fit(train_gen,epochs=20,validation_data=test_gen)

Epoch 1/20
  7/625 [..............................] - ETA: 7:10 - loss: 32.6339 - mae: 32.6339

KeyboardInterrupt: 